In [1]:
#dependencies
import os
import h5py
import pylab as pl
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy as scipy

import utils.conversions as cnv
import utils.plotutils as plu
import utils.movie_plotutils as mplu
import utils.imreadin as imr
import utils.readMov as rdm
import utils.movie_aec as aec
import utils.train_model as tm

import imp
imp.reload(rdm)
imp.reload(tm)
imp.reload(cnv)
imp.reload(mplu)
from random import shuffle

%matplotlib inline
%config InlineBackend.print_figure_kwargs = {'dpi' : 500} #plotting pretty

/home/vasha/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
imp.reload(rdm)
movie_name = 'snow_monkeys' #'snow_monkeys' #
pixelpatchsize = 6
framepatchsize = 6

colorinvert = False

#m = imr.load_images('vh_corr', patch_edge_size=8)
#del m
with tf.device('/cpu:0'):
    try:
        m
        print('Movie Already Loaded.')
    except NameError:
        m = rdm.get_movie(movie_name,
                          pixel_patch_size = pixelpatchsize,
                          frame_patch_size = framepatchsize
                          )
        #m = np.transpose(m, (0, 3, 1, 2)) #change axis to [batchsize, time_patchsize, x_patchsize, y_patchsize]

        print("Movie Loaded. Shape is " + str(np.shape(m)))
        
        #only make anew dict and change these in params dict if we have reloaded images.
        params = {} #make a new dictionary
        #fill up our new dictionary
        params["movie_name"] = movie_name
        params['nmovies'] = np.shape(m)[0]
        params["pixelpatchsize"] = pixelpatchsize
        params["framepatchsize"] = framepatchsize
print("Done")

(120, 676, 1280)
normalizing movie...
making patches...
(477120, 6, 6, 6)
Movie Loaded. Shape is (477120, 6, 6, 6)
Done


In [ ]:
#fixed parameters
params['frames_per_channel'] = params["framepatchsize"]  #OR convolution window (number of frames that project on one channel)
params['clipvec_len'] = params['frames_per_channel']*params['pixelpatchsize']*params['pixelpatchsize']

params["batchsize"] = 10000
params["learning_rate"] = 0.001

params["iterations"] = np.int(np.floor(params["nmovies"]/params["batchsize"]))
params["epochs"] = 10000


params['nonlinearity'] = 'relu'

params['nneurons'] = 400
#pixelpatchsize**2 * framepatchsize

#params for sweeping
noise_xs_rs_pairs = [[0.,0.]] #, [1e-1, 5], [1e-1, 2], [1e-1, 3.5], [1e-3,5e-1]]


#check to make sure our training numbers are consistent with our dataset
params["moviesperepoch"] = params["iterations"] *params["batchsize"]
if(params["moviesperepoch"] > params['nmovies']):
    print('ERROR! Trying to train on', params["moviesperepoch"],'movies per epoch, but we only have',params['nmovies'],'movie clips!')
else:
    print('Training',params["moviesperepoch"],'out of',params["nmovies"],'total movies.')
    
print('Compression Level: {}'.format((params["pixelpatchsize"]**2*params['frames_per_channel'])/params["nneurons"]))

Training 470000 out of 477120 total movies.
Compression Level: 0.54


In [ ]:
#reload
imp.reload(aec)
imp.reload(tm)
imp.reload(mplu)
imp.reload(plu)

for xs,rs in noise_xs_rs_pairs:
    params['noise_x'] = xs
    params['noise_r'] = rs

    #make our model
    aecmodel = aec.movie_aec_model(params)

    #train it
    [cost_evolution,
     activation_evolution,
     inweights_evolution,
     outweights_evolution,
     clips,
     recons,
     final_inweights,
     final_outweights,
     final_activations] = tm.train_movie_model(aecmodel, m)

    #mplu.save_plots(aecmodel,
    #                cost_evolution,
    #                activation_evolution,
    #                inweights_evolution,
    #                outweights_evolution,
    #                clips,
    #                recons,
    #                final_inweights,
    #                final_outweights,
    #                final_activations)

    print('All Done!')

Compressing by 0.09 for a total of 400 neurons
400 hidden neurons, noise_in at 0.0, noise_out at 0.0
Training 47 iterations in 10000 epochs... 


Epoch 0/10000 Epoch 1/10000 Epoch 2/10000 Epoch 3/10000 Epoch 4/10000 Epoch 5/10000 Epoch 6/10000 Epoch 7/10000 Epoch 8/10000 Epoch 9/10000 Epoch 10/10000 Epoch 11/10000 Epoch 12/10000 Epoch 13/10000 Epoch 14/10000 Epoch 15/10000 Epoch 16/10000 Epoch 17/10000 Epoch 18/10000 Epoch 19/10000 Epoch 20/10000 Epoch 21/10000 Epoch 22/10000 Epoch 23/10000 Epoch 24/10000 Epoch 25/10000 Epoch 26/10000 Epoch 27/10000 Epoch 28/10000 Epoch 29/10000 Epoch 30/10000 Epoch 31/10000 Epoch 32/10000 Epoch 33/10000 Epoch 34/10000 Epoch 35/10000 Epoch 36/10000 Epoch 37/10000 Epoch 38/10000 Epoch 39/10000 Epoch 40/10000 Epoch 41/10000 Epoch 42/10000 Epoch 43/10000 Epoch 44/10000 Epoch 45/10000 Epoch 46/10000 Epoch 47/10000 Epoch 48/10000 Epoch 49/10000 Epoch 50/10000 Epoch 51/10000 Epoch 52/10000 Epoch 53/10000 Epoch 54/10000 Epoch 55/10000 Epoch 56/10000 Epoch 57/10000 Epoch 58/10000 Epoch 59/10000 Epoch 60/10000 Epoch 61/10000 Epoch 62/10000 Epoch 63/10000 Epoch 64/10000 Epoch 65/10000 Epoch 66/10000 Epoch

Epoch 519/10000 Epoch 520/10000 Epoch 521/10000 Epoch 522/10000 Epoch 523/10000 Epoch 524/10000 Epoch 525/10000 Epoch 526/10000 Epoch 527/10000 Epoch 528/10000 Epoch 529/10000 Epoch 530/10000 Epoch 531/10000 Epoch 532/10000 Epoch 533/10000 Epoch 534/10000 Epoch 535/10000 Epoch 536/10000 Epoch 537/10000 Epoch 538/10000 Epoch 539/10000 Epoch 540/10000 Epoch 541/10000 Epoch 542/10000 Epoch 543/10000 Epoch 544/10000 Epoch 545/10000 Epoch 546/10000 Epoch 547/10000 Epoch 548/10000 Epoch 549/10000 Epoch 550/10000 Epoch 551/10000 Epoch 552/10000 Epoch 553/10000 Epoch 554/10000 Epoch 555/10000 Epoch 556/10000 Epoch 557/10000 Epoch 558/10000 Epoch 559/10000 Epoch 560/10000 Epoch 561/10000 Epoch 562/10000 Epoch 563/10000 Epoch 564/10000 Epoch 565/10000 Epoch 566/10000 Epoch 567/10000 Epoch 568/10000 Epoch 569/10000 Epoch 570/10000 Epoch 571/10000 Epoch 572/10000 Epoch 573/10000 Epoch 574/10000 Epoch 575/10000 Epoch 576/10000 Epoch 577/10000 Epoch 578/10000 Epoch 579/10000 Epoch 580/10000 Epoch 58

Epoch 1030/10000 Epoch 1031/10000 Epoch 1032/10000 Epoch 1033/10000 Epoch 1034/10000 Epoch 1035/10000 Epoch 1036/10000 Epoch 1037/10000 Epoch 1038/10000 Epoch 1039/10000 Epoch 1040/10000 Epoch 1041/10000 Epoch 1042/10000 Epoch 1043/10000 Epoch 1044/10000 Epoch 1045/10000 Epoch 1046/10000 Epoch 1047/10000 Epoch 1048/10000 Epoch 1049/10000 Epoch 1050/10000 Epoch 1051/10000 Epoch 1052/10000 Epoch 1053/10000 Epoch 1054/10000 Epoch 1055/10000 Epoch 1056/10000 Epoch 1057/10000 Epoch 1058/10000 Epoch 1059/10000 Epoch 1060/10000 Epoch 1061/10000 Epoch 1062/10000 Epoch 1063/10000 Epoch 1064/10000 Epoch 1065/10000 Epoch 1066/10000 Epoch 1067/10000 Epoch 1068/10000 Epoch 1069/10000 Epoch 1070/10000 Epoch 1071/10000 Epoch 1072/10000 Epoch 1073/10000 Epoch 1074/10000 Epoch 1075/10000 Epoch 1076/10000 Epoch 1077/10000 Epoch 1078/10000 Epoch 1079/10000 Epoch 1080/10000 Epoch 1081/10000 Epoch 1082/10000 Epoch 1083/10000 Epoch 1084/10000 Epoch 1085/10000 Epoch 1086/10000 Epoch 1087/10000 Epoch 1088/100

Epoch 1512/10000 Epoch 1513/10000 Epoch 1514/10000 Epoch 1515/10000 Epoch 1516/10000 Epoch 1517/10000 Epoch 1518/10000 Epoch 1519/10000 Epoch 1520/10000 Epoch 1521/10000 Epoch 1522/10000 Epoch 1523/10000 Epoch 1524/10000 Epoch 1525/10000 Epoch 1526/10000 Epoch 1527/10000 Epoch 1528/10000 Epoch 1529/10000 Epoch 1530/10000 Epoch 1531/10000 Epoch 1532/10000 Epoch 1533/10000 Epoch 1534/10000 Epoch 1535/10000 Epoch 1536/10000 Epoch 1537/10000 Epoch 1538/10000 Epoch 1539/10000 Epoch 1540/10000 Epoch 1541/10000 Epoch 1542/10000 Epoch 1543/10000 Epoch 1544/10000 Epoch 1545/10000 Epoch 1546/10000 Epoch 1547/10000 Epoch 1548/10000 Epoch 1549/10000 Epoch 1550/10000 Epoch 1551/10000 Epoch 1552/10000 Epoch 1553/10000 Epoch 1554/10000 Epoch 1555/10000 Epoch 1556/10000 Epoch 1557/10000 Epoch 1558/10000 Epoch 1559/10000 Epoch 1560/10000 Epoch 1561/10000 Epoch 1562/10000 Epoch 1563/10000 Epoch 1564/10000 Epoch 1565/10000 Epoch 1566/10000 Epoch 1567/10000 Epoch 1568/10000 Epoch 1569/10000 Epoch 1570/100

In [ ]:
print(final_inweights.shape)
print(final_outweights.shape)

In [ ]:
plt.hist(final_outweights.flatten())

In [ ]:
plt.plot(np.log(np.mean(cost_evolution,axis=2).flatten()))

In [ ]:
#test = final_outweights.T
#print(np.shape(test))
print(final_outweights.shape)
mplu.plot_temporal_weights(final_outweights[...,0].T);

In [ ]:
imp.reload(mplu)
imp.reload(plu)
#plu.plot_tiled_rfs(np.rollaxis(inweights_evolution[-1][0,:,:,:],2,0))
#mplu.plot_temporal_weights(np.rollaxis(inweights_evolution[-1][0,:,:,:],2,0))

#print(inweights_evolution.shape)
#test = inweights_evolution[-1].T
test = outweights_evolution[-1]
print(test.shape)
#test = np.rollaxis(test[0,:,:,:],2,0)
plu.plot_tiled_rfs(test)
#print(test)
#mplu.plot_temporal_weights(final_outweights);
#plu.plot_tiled_rfs()

In [ ]:
print(final_inweights.shape)
print(final_outweights.shape)
plu.plot_tiled_rfs(final_inweights[-1].T)
plt.show()
plu.plot_tiled_rfs(final_outweights[...,-1])

In [ ]:
imp.reload(mplu)
imp.reload(plu)
print(final_inweights.shape)
#mplu.plot_temporal_weights(final_inweights);
#plu.plot_tiled_rfs(np.rollaxis(final_inweights[0,:,:,:],2,0))

#plt.figure(figsize=(10,3))
#test = np.transpose(final_inweights,axes=(0,2,1,3))
#print(test.shape)
#mplu.plot_temporal_weights(final_outweights.T);
mplu.plot_temporal_weights(final_inweights);
#plu.plot_tiled_rfs(np.rollaxis(test[0,:,:,:],2,0))


In [ ]:
imp.reload(mplu)
animation = mplu.weights_movieclip(final_outweights.T);
#animation.ipython_display(fps=20, loop=True, autoplay=True)
#from IPython.display import HTML
#HTML('<img src="./weight_movie.gif>')

In [ ]:
print(final_inweights[-1].shape)
p = plu.pad_data(final_inweights[-1].T)
plt.imshow(p,cmap='Greys_r')

In [ ]:
mplu.plot_temporal_weights(np.transpose(final_inweights,axes=(0,2,1,3)));

In [ ]:
##TWO BASIS VECTORS##
mplu.plot_mov_recon(final_outweights.T[:,:,:,8],final_outweights.T[:,:,:,20],nframes=params['frames_per_channel']);


In [ ]:
##TWO BASIS VECTORS##
mplu.plot_mov_recon(test[2,:,:,:],final_inweights[2,:,:,:],nframes=params['frames_per_channel']);


In [ ]:
#diff = final_inweights - inweights_evolution[1]
#print(diff.shape)
print(final_inweights.shape)
for i in range(final_inweights.shape[3]):
    sbplt = np.sqrt(final_inweights.shape[3])
    plt.subplot(sbplt,sbplt,i+1)
    plt.imshow(np.linalg.inv(final_outweights.T[0,:,:,i]),cmap='Greys_r')
    #plt.imshow(final_inweights[2,:,:,i],cmap='Greys_r')
    #plt.imshow(test,cmap='Greys_r')
    plt.xticks([])
    plt.yticks([])


In [ ]:
print(final_inweights.shape)
print(final_outweights.shape)
print(np.array(recons).shape)
print(np.array(clips).shape)

In [ ]:
randomclipindex = np.random.randint(np.shape(clips[0])[0])

#print the cost of this instance
print(np.mean(np.log1p(clips[-1][randomclipindex,:,:,:] - recons[-1][randomclipindex,:,:,:])))

mplu.plot_mov_recon(clips[-1][randomclipindex,:,:,:],recons[-1][randomclipindex,:,:,:],nframes=5);

In [ ]:
##Check norm of weights
plt.plot(np.linalg.norm(final_inweights.reshape(8*64,1000), ord=2, axis=0))

In [ ]:
print(inweights_evolution[-1].T.shape)
#plu.plot_tiled_rfs(inweights_evolution[-1].T[:,:,:,0])
padded_wv = plu.pad_data(inweights_evolution[-1].T[:,:,:,0])
print(padded_wv.shape)
plt.imshow(padded_wv)

In [ ]:
import imageio
images = []

for i in range(np.shape(final_inweights)[3]):
    basisfun = plt.imshow(final_inweights[0,:,:,i],cmap='Greys_r');
    images.append(basisfun)
imageio.mimsave('test.gif', images)

In [ ]:
p = plt.errorbar(np.arange(len(cost_evolution)),
                 np.log(np.mean(np.array(cost_evolution),axis=1)),
                 yerr=np.log(np.std(np.array(cost_evolution),axis=1)));
plt.title("Log-Cost Evolution Over Iterations (Errorbar = 1SD)")

In [ ]:
np.shape(np.array(activation_evolution))

In [ ]:
p = plt.errorbar(np.arange(len(activation_evolution)),
                 np.array(activation_evolution),
                 yerr=np.std(np.array(activation_evolution),axis=0));
plt.title('Evolution of Activation over Learning')

In [ ]:
plt.imshow(recons[0][randomclipindex],cmap="Greys_r")
plt.colorbar()

In [ ]:
plt.imshow(clips[0][randomclipindex],cmap="Greys_r")
plt.colorbar()

In [ ]:

np.shape(final_inweights)

In [ ]:
np.shape(inweights_evolution)

In [ ]:
import sklearn as skl

npatches = 1000

test = m[0:npatches,:5,:,:].reshape((npatches,-1))
print(test.shape)
cov_mat = np.cov(test)
eig_val, eig_vec = np.linalg.eigh(cov_mat)
print(eig_val)
print(np.shape(eig_val))

invcovmat = eig_vec
#recon_val, recon_vec = np.linalg.eig(invcovmat)

#recon_vec

#pca = skl.decomposition.PCA()
#pca.fit(test)
#np.pca(test).shape
#pca